In [ ]:
from qiskit import *
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.visualization import plot_histogram, plot_bloch_vector
from qiskit.tools.visualization import plot_bloch_multivector
from pandas import read_csv
from math import sqrt, pi
import numpy as np
import requests
import io


#   Load dataframe file

# The imported file was made using data of 2018 FIFA World Cup taken from statsbombspy library
url = "https://raw.githubusercontent.com/AdrFelHB/UPTC_Qiskit_Fallfest/main/teams_coef.csv"
dataframe_file = requests.get(url).content

df = read_csv(io.StringIO(dataframe_file.decode('utf-8')),index_col=False)

#   Put columns of dataframe in separated list
Teams = list(df.Team)
Scores = list(df.coef)


#   Command Line Interface
 
print("Welcome to Quantum Soccer Predicter\n")
print("Please choose two teams from the following list and ¡get ready to bet!\n\n")

for line in range(0,len(Teams)):
        string= ''.join(str(line) + '.\t'+ Teams[line])
        print(string)

#    Ask user to enter teams which it is interested in

Home =  int (input("Enter the home team index: "))
Away =  int (input("Enter the away team index: "))

print(f"\nYou Chose:\nHome: {Teams[Home]} \t&\t Away: {Teams[Away]}")

#     Ask for advantages

advantage_team = int(input(f"Weather is going to benefit any team\n1. {Teams[Home]}\t2. {Teams[Away]}\t3. no teams is benefited\n"))
if advantage_team == 1:
  rotation_1 = pi/16
elif advantage_team == 2:
  rotation_1 = pi
elif advantage_team == 3:
  rotation_1 = 0

#   Compute coefficients for both teams

#   rate for Home team
rate_home = Scores[Home]

#   rate for Away team
rate_away = Scores[Away]

#   computation of normalized coefficients
coefficient_home = (rate_home)/sqrt((rate_home**2) + (rate_away**2))
coefficient_away = (rate_away)/sqrt((rate_home**2) + (rate_away**2))


initial_state = [coefficient_away, coefficient_home]#define el esta. inicial como |q_0>


qr = QuantumRegister(1)  
cr = ClassicalRegister(1)
qc = QuantumCircuit(qr,cr)

# Declare initial_state as |q_0>

qc.initialize(initial_state, 0 )
simulator = Aer.get_backend('statevector_simulator')
result = execute(qc, backend = simulator).result()
statevector = result.get_statevector()

In [ ]:
%matplotlib inline
qc.draw(initial_state=True, cregbundle=False,output='mpl')

plot_bloch_multivector (statevector)

In [ ]:
qc.u(rotation_1, 0, 0, 0)

qc.draw(initial_state=True, cregbundle= False,output='mpl')

In [ ]:
simulator = Aer.get_backend('statevector_simulator')

result = execute(qc, backend = simulator).result()

statevector = result.get_statevector()
plot_bloch_multivector (statevector)

In [ ]:

qc.measure(qr,cr)

qc.draw(initial_state=True, cregbundle= False,output='mpl')

In [ ]:
simulator = Aer.get_backend('qasm_simulator')

result= execute(qc,backend = simulator, shots = 1024).result()

plot_histogram(result.get_counts(qc))

In [ ]:
qc.u(pi, 0, 0, 0)

qc.draw(initial_state=True, cregbundle= False,output='mpl')

In [ ]:
qc.measure(qr,cr)

qc.draw(initial_state=True, cregbundle= False,output='mpl')

In [ ]:
simulator = Aer.get_backend('qasm_simulator')

result = execute(qc,backend = simulator, shots = 1024).result()

plot_histogram(result.get_counts(qc))

In [ ]:

# final part

probability_home =result.get_counts(qc)['1']/1000
probability_away = result.get_counts(qc)['0']/1000



# defining c as one probability divide in the other one

c = probability_away/probability_home
print(c)

if (c > 0.9) and (c < 1.1):
  print(f"There is tie between {Teams[Home]} and {Teams[Away]}")
elif (c < 0.9):
  if (c > 0.7) and (c < 0.9):
    print(f"team winer is {Teams[Away]} for 1 goal difference")
  elif  (c > 0.5) and (c < 0.7):
    print(f"team winer is {Teams[Away]} for 2 goal difference")
  elif  (c > 0) and (c < 0.5):
    print(f"team winer is {Teams[Away]} for more than 3 goal difference")
elif (c > 1.1):
  if (c > 1.1) and (c < 1.3):
    print(f"team winer is {Teams[Home]} for 1 goal difference")
  elif (c > 1.3) and (c < 1.5):
    print(f"team winer is {Teams[Home]} for 2 goal difference")
  elif (c > 1.5):
    print(f"team winer is {Teams[Home]} for more than 3 goal difference")